<a href="https://colab.research.google.com/github/SayhellotoAI/NLP-progress/blob/master/bert_mrc_ner2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gdrive_path = "/drive/My Drive/colab/bert-ner"

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [2]:
import sys
sys.path.append('/drive/My Drive/colab/bert-ner/mrc-for-flat-nested-ner')

In [3]:
!pip install future
!pip install tensorboard
!pip install transformers
#!pip install transformers -i https://pypi.python.org/simple
!pip install -r '/drive/My Drive/colab/bert-ner/mrc-for-flat-nested-ner/requirements.txt'

     |████████████████████████████████| 1.4MB 9.1MB/s 
     |████████████████████████████████| 2.9MB 29.9MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=bb4606b16eebba16e761c05c13f121753eb778bb813384992f4c73c54fd71a6b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 409kB 8.4MB/s 
     |████████████████████████████████| 276kB 13.9MB/s 
     |████████████████████████████████| 2.8MB 18.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=578335dcdfb37c784e6599abab0247729439d99f7fb97bbe41fc2db3fc35acae
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
ERROR: tensorflow 2.3.0 has requirement tensorboard<3,>=2.3.0, but you'll 

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import copy
import json
import logging
import math
import sys
import collections
import os
import unicodedata
import random
import string
import re
import numpy as np

from io import open
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

from torch.nn import CrossEntropyLoss, Dropout, Embedding, Softmax
from torch.nn.parameter import Parameter
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)
from tqdm.autonotebook import tqdm, trange


SEED = 81


max_seq_length = 512 #@param {type: "integer"}
doc_stride = 128 #@param {type: "integer"}
max_query_length = 96 #@param {type: "integer"}
max_answer_length = 30 #@param {type: "integer"}
n_best_size = 57 #@param {type: "integer"}

train_batch_size = 2 #@param {type: "integer"}
learning_rate = 5e-5 #@param {type:"raw"}
warmup_proportion = 0.1 #@param {type:"raw"}
num_train_epochs = 20 #@param {type:"raw"}

max_grad_norm = 1.0 #@param {type:"raw"}
adam_epsilon = 1e-6 #@param {type:"raw"}
weight_decay = 0.01 #@param {type:"raw"}

gdrive_path = "/drive/My Drive/colab/bert-ner/bert_small"
output_dir = gdrive_path
checkpoint = os.path.join(gdrive_path, "bert_small_ckpt.bin")
model_config = os.path.join(gdrive_path, "bert_small_NER.json")
train_file = os.path.join(gdrive_path, "KorQuAD_v1.0_train.json")
vocab_file = os.path.join(gdrive_path, "ko_vocab_32k.txt")

In [5]:
#@title BertTokenizer (더블클릭하여 내용 확인 가능합니다.)

logger = logging.getLogger(__name__)

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    index = 0
    with open(vocab_file, "r", encoding="utf-8") as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab


def whitespace_tokenize(text):
    """Runs basic whitespace cleaning and splitting on a piece of text."""
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens


class BertTokenizer(object):
    """Runs end-to-end tokenization: punctuation splitting + wordpiece"""

    def __init__(self, vocab_file, do_lower_case=False, max_len=None, do_basic_tokenize=True,
                 never_split=("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]","PS","LC","OG","AF","DT","TI","CV","AM","PT","QT","FD","TR","EV","MT","TM"
)):
        """Constructs a BertTokenizer.

        Args:
          vocab_file: Path to a one-wordpiece-per-line vocabulary file
          do_lower_case: Whether to lower case the input
                         Only has an effect when do_wordpiece_only=False
          do_basic_tokenize: Whether to do basic tokenization before wordpiece.
          max_len: An artificial maximum length to truncate tokenized sequences to;
                         Effective maximum length is always the minimum of this
                         value (if specified) and the underlying BERT model's
                         sequence length.
          never_split: List of tokens which will never be split during tokenization.
                         Only has an effect when do_wordpiece_only=False
        """
        if not os.path.isfile(vocab_file):
            raise ValueError(
                "Can't find a vocabulary file at path '{}'. To load the vocabulary from a Google pretrained "
                "model use `tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)`".format(vocab_file))
        self.vocab = load_vocab(vocab_file)
        self.ids_to_tokens = collections.OrderedDict(
            [(ids, tok) for tok, ids in self.vocab.items()])
        self.do_basic_tokenize = do_basic_tokenize
        if do_basic_tokenize:
            self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case,
                                                  never_split=never_split)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)
        self.max_len = max_len if max_len is not None else int(1e12)

    def tokenize(self, text):
        split_tokens = []
        if self.do_basic_tokenize:
            for token in self.basic_tokenizer.tokenize(text):
                for sub_token in self.wordpiece_tokenizer.tokenize(token):
                    split_tokens.append(sub_token)
        else:
            split_tokens = self.wordpiece_tokenizer.tokenize(text)
        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        """Converts a sequence of tokens into ids using the vocab."""
        ids = []
        for token in tokens:
            ids.append(self.vocab[token])
        if len(ids) > self.max_len:
            logger.warning(
                "Token indices sequence length is longer than the specified maximum "
                " sequence length for this BERT model ({} > {}). Running this"
                " sequence through BERT will result in indexing errors".format(len(ids), self.max_len)
            )
        return ids

    def convert_ids_to_tokens(self, ids):
        """Converts a sequence of ids in wordpiece tokens using the vocab."""
        tokens = []
        for i in ids:
            tokens.append(self.ids_to_tokens[i])
        return tokens


class BasicTokenizer(object):
    """Runs basic tokenization (punctuation splitting, lower casing, etc.)."""

    def __init__(self,
                 do_lower_case=True,
                 never_split=("[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]")):
        """Constructs a BasicTokenizer.

        Args:
          do_lower_case: Whether to lower case the input.
        """
        self.do_lower_case = do_lower_case
        self.never_split = never_split

    def tokenize(self, text):
        """Tokenizes a piece of text."""
        text = self._clean_text(text)
        text = self._tokenize_chinese_chars(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if self.do_lower_case and token not in self.never_split:
                token = self._run_strip_accents(token)
            token = token.lower()
            split_tokens.extend(self._run_split_on_punc(token))

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        """Strips accents from a piece of text."""
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text):
        """Splits punctuation on a piece of text."""
        if text in self.never_split:
            return [text]
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]

    def _tokenize_chinese_chars(self, text):
        """Adds whitespace around any CJK character."""
        output = []
        for char in text:
            cp = ord(char)
            if self._is_chinese_char(cp):
                output.append(" ")
                output.append(char)
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)

    def _is_chinese_char(self, cp):
        """Checks whether CP is the codepoint of a CJK character."""
        # This defines a "chinese character" as anything in the CJK Unicode block:
        #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
        #
        # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
        # despite its name. The modern Korean Hangul alphabet is a different block,
        # as is Japanese Hiragana and Katakana. Those alphabets are used to write
        # space-separated words, so they are not treated specially and handled
        # like the all of the other languages.
        if ((cp >= 0x4E00 and cp <= 0x9FFF) or  #
                (cp >= 0x3400 and cp <= 0x4DBF) or  #
                (cp >= 0x20000 and cp <= 0x2A6DF) or  #
                (cp >= 0x2A700 and cp <= 0x2B73F) or  #
                (cp >= 0x2B740 and cp <= 0x2B81F) or  #
                (cp >= 0x2B820 and cp <= 0x2CEAF) or
                (cp >= 0xF900 and cp <= 0xFAFF) or  #
                (cp >= 0x2F800 and cp <= 0x2FA1F)):  #
            return True

        return False

    def _clean_text(self, text):
        """Performs invalid character removal and whitespace cleanup on text."""
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)


class WordpieceTokenizer(object):
    """Runs WordPiece tokenization."""

    def __init__(self, vocab, unk_token="[UNK]", max_input_chars_per_word=100):
        self.vocab = vocab
        self.unk_token = unk_token
        self.max_input_chars_per_word = max_input_chars_per_word

    def tokenize(self, text):
        """Tokenizes a piece of text into its word pieces.

        This uses a greedy longest-match-first algorithm to perform tokenization
        using the given vocabulary.

        For example:
          input = "unaffable"
          output = ["un", "##aff", "##able"]

        Args:
          text: A single token or whitespace separated tokens. This should have
            already been passed through `BasicTokenizer`.

        Returns:
          A list of wordpiece tokens.
        """

        output_tokens = []
        for token in whitespace_tokenize(text):
            chars = list(token)
            if len(chars) > self.max_input_chars_per_word:
                output_tokens.append(self.unk_token)
                continue

            is_bad = False
            start = 0
            sub_tokens = []
            while start < len(chars):
                end = len(chars)
                cur_substr = None
                while start < end:
                    substr = "".join(chars[start:end])
                    if start > 0:
                        substr = "##" + substr
                    if substr in self.vocab:
                        cur_substr = substr
                        break
                    end -= 1
                if cur_substr is None:
                    is_bad = True
                    break
                sub_tokens.append(cur_substr)
                start = end

            if is_bad:
                output_tokens.append(self.unk_token)
            else:
                output_tokens.extend(sub_tokens)
        return output_tokens


def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False


def _is_control(char):
    """Checks whether `chars` is a control character."""
    # These are technically control characters but we count them as whitespace
    # characters.
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False


def _is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False


In [6]:
#@title MRC2NER Input Dataset (더블클릭하여 내용 확인 가능합니다.)
from torch.utils.data import Dataset
class MRC2NERInput(object):
    def __init__(self, tokens, query, entity_label, impossible=True, end_position=[], span_position=[], start_position=[], qas_id="0.0"):
        self.tokens = tokens
        self.end_position = end_position
        self.entity_label = entity_label
        self.impossible = impossible
        self.query = query
        self.qas_id = qas_id
        self.span_position = span_position
        self.start_position = start_position

labels = {"PS":"사람은 실존인물, 신화속 인물, 게임/소설의 가상 캐릭터 등을 나타낸다.",
          "LA":"지역/장소와 지형/지리 명칭 등을 모두 포함한다. 지역/장소는 장소 계열로 표시하고, 지형/지리 명칭은 지리 계열로 표시한다.",
          "OG":"기관/단체 명칭을 나타내며, 경제기관, 교육기관, 군사기관, 미디어/방송기관, 스포츠기관, 예술기관, 의료기관, 종교기관, 과학기관, 도서관, 법률기관, 정치/정부기관, 음식 관련 기관, 숙박기관 등으로 세분화된다.",
          "AF":"인공물을 나타낸다. 인공물이란 사람에 의해 창조된 대상물을 말하며 문화재, 건물, 악기, 도로, 무기, 운송수단, 작품명 등이 이에 해당한다. 유의할 점은 인공물 내부에 포함된 세부범주에 해당되지 않는 인공물은 태깅 대상에서 배제한다는 점이다.",
          "DT":"날짜 표현을 나타낸다",
          "TI":"시간 표현을 나타낸다",
          "CV":"문명/문화와 관련된 용어들로 구성되어 있다",
          "AM":"생물은 진핵생물 중에서 동물에 해당되는 것으로, 사람을 제외한 짐승을 대상으로 한다.",
          "PT":"식물 명칭을 나타낸다.",
          "QT":"수량 표현을 나타낸다.",
          "FD":"학문 분야를 나타낸다. 세부 범주로는 인문학, 사회과학, 자연과학, 응용과학으로 구성되어 있다. 더불어 학문 분야는 학파를 포함한다. 과학, 사회과학, 의학, 예술, 철학 관련 학파 및 유파들은 학문분야의 세부 범주로 명시된다.",
          "TR":"이론은 특정 이론, 법칙, 원리 등의 명칭을 의미한다. 이론은 명확히 기술될 수 있는 고유의 이론, 법칙, 원리만을 말하며, 일반적인 용어들은 해당하지 않는다.",
          "EV":"특정 사건/사고 명칭을 나타낸다.",
          "MT":"물질에 대한 표현을 나타낸다.",
          "TM":"용어를 나타낸다. 색, 방향, 기후지역, 모양, 세포 명칭과 생물의 조직 및 기관, 증상/증세/질병, 약, 전자기기 등 "
}
label_to_idx = {f:i for i, f in enumerate(labels)}
class MRCNERDataset(Dataset):
    def __init__(self, tokenizer, read_data, possible_only=True):
        self.read_data = read_data

        self.tokenizer = tokenizer
        self.max_length = max_seq_length
        self.possible_only = possible_only

        alldata = []
        for (example_index, text) in enumerate(self.read_data):
            text = re.sub(':LC', ':LA', text)
            x = tokenizer.tokenize(text)

            del_list = []
            y = []
            j = 0
            onedata = []
            for i in labels:
                onedata.append(MRC2NERInput(tokens = text,query=labels[i],entity_label=i,impossible=True,end_position=[],span_position=[],start_position=[],qas_id=str(example_index)+"."+str(label_to_idx[i])))
                #context, end_position=[], entity_label, impossible=True, query, span_position=[], start_position=[]

            for i in range(len(x)):
                if i < j:
                    continue
                if x[i] == "<":
                    for j in range(i,len(x)):
                        if x[j] == ':' and x[j+2]==">":
                            label = x[j+1]
                            del_list.append(i)
                            del_list.extend(list(range(j,j+3)))
                            break

                    onedata[label_to_idx[label.upper()]].start_position.append(len(y))
                    onedata[label_to_idx[label.upper()]].impossible = False
                    y.append(1)

                    k = i+1
                    while k+1 < j:
                        y.append(2)
                        k += 1

                    onedata[label_to_idx[label.upper()]].end_position.append(len(y)-1)
                    onedata[label_to_idx[label.upper()]].span_position.append([onedata[label_to_idx[label.upper()]].start_position[-1],len(y)-1])    
                    j = j+3
                else:
                    y.append(0)

            del_list.sort(reverse=True)
            for d in del_list:
                del x[d]
            
            for i in range(len(labels)):
                onedata[i].tokens = x

            alldata = alldata + onedata
        
        ## if impossible
        alldata = [
            x for x in alldata if x.start_position
        ]
        self.alldata = alldata
            
    def __len__(self):
        return len(self.alldata)
    
    def __getitem__(self, item):
        """
        Args:
            item: int, idx
        Returns:
            tokens: tokens of query + context, [seq_len]
            token_type_ids: token type ids, 0 for query, 1 for context, [seq_len]
            start_labels: start labels of NER in tokens, [seq_len]
            end_labels: end labels of NER in tokens, [seq_len]
            label_mask: label mask, 1 for counting into loss, 0 for ignoring. [seq_len]
            match_labels: match labels, [seq_len, seq_len]
            sample_idx: sample id
            label_idx: label id
        """
        data = self.alldata[item]
        tokenizer = self.tokenizer

        query = tokenizer.tokenize(data.query)

        x = ["[CLS]"] + query + ["[SEP]"] + data.tokens + ["[SEP]"]
        new_start_positions = [start+2+len(query) for start in data.start_position]
        new_end_positions = [end+2+len(query) for end in data.end_position]
        
        if len(x) > max_seq_length:
            x = query_tokens[0:max_seq_length]
            print("x exceed max_seq_length",len(x))

        id_tokens = tokenizer.convert_tokens_to_ids(x)
        token_type_ids = [[0] * (len(query)+2)] + [[1] * (len(data.tokens)+1)]
        token_type_ids = sum(token_type_ids, [])
        
        match_labels = torch.zeros([max_seq_length, max_seq_length], dtype=torch.long)
        for start, end in zip(new_start_positions, new_end_positions):
            if start >= max_seq_length or end >= max_seq_length:
                continue
            match_labels[start, end] = 1
        
        sample_idx, label_idx = data.qas_id.split(".")
        sample_idx = torch.LongTensor([int(sample_idx)])
        label_idx = torch.LongTensor([int(label_idx)])   

        start_labels = [(1 if idx in new_start_positions else 0)
                        for idx in range(len(id_tokens))]
        end_labels = [(1 if idx in new_end_positions else 0)
                      for idx in range(len(id_tokens))]

        #print(x)
        #print(id_tokens)
        #print(token_type_ids)
        #print(new_start_positions)
        #print(new_end_positions)
        #print(start_labels)
        #print(end_labels)
        #print(len(id_tokens),len(token_type_ids))
        #print(len(start_labels),len(end_labels))

        while len(id_tokens) < max_seq_length:
            id_tokens.append(0)
            token_type_ids.append(0)
            start_labels.append(0)
            end_labels.append(0)

        
        return [
            torch.LongTensor(id_tokens),
            torch.LongTensor(token_type_ids),
            torch.LongTensor(start_labels),
            torch.LongTensor(end_labels),
            torch.LongTensor(token_type_ids),#start_label_mask
            torch.LongTensor(token_type_ids),#end _label_mask
            match_labels,
            sample_idx,
            label_idx
        ]
        #tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, sample_idx, label_idx = batch
    
    @staticmethod
    def get_test(text,tokenizer,query):
        x = ["[CLS]"] + tokenizer.tokenize(labels[query]) + ["[SEP]"] + tokenizer.tokenize(text) + ["[SEP]"]
        id_tokens = tokenizer.convert_tokens_to_ids(x)
        token_type_ids = [[0] * (len(tokenizer.tokenize(labels[query]))+2)] + [[1] * (len(tokenizer.tokenize(text))+1)]
        token_type_ids = sum(token_type_ids, [])
        
        while len(id_tokens) < max_seq_length:
            id_tokens.append(0)
            token_type_ids.append(0)

        print(x)
        print(id_tokens)
        print(token_type_ids)
        return [
             torch.LongTensor(id_tokens),  
             torch.LongTensor(token_type_ids),
             x
        ]
         



In [7]:
#@title BERT Optimizer & Scheduler (더블클릭하여 내용 확인 가능합니다.)
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""PyTorch optimization for BERT model."""

logger = logging.getLogger(__name__)

class ConstantLRSchedule(LambdaLR):
    """ Constant learning rate schedule.
    """
    def __init__(self, optimizer, last_epoch=-1):
        super(ConstantLRSchedule, self).__init__(optimizer, lambda _: 1.0, last_epoch=last_epoch)


class WarmupConstantSchedule(LambdaLR):
    """ Linear warmup and then constant.
        Linearly increases learning rate schedule from 0 to 1 over `warmup_steps` training steps.
        Keeps learning rate schedule equal to 1. after warmup_steps.
    """
    def __init__(self, optimizer, warmup_steps, last_epoch=-1):
        self.warmup_steps = warmup_steps
        super(WarmupConstantSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1.0, self.warmup_steps))
        return 1.


class WarmupLinearSchedule(LambdaLR):
    """ Linear warmup and then linear decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Linearly decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps.
    """
    def __init__(self, optimizer, warmup_steps, t_total, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        super(WarmupLinearSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1, self.warmup_steps))
        return max(0.0, float(self.t_total - step) / float(max(1.0, self.t_total - self.warmup_steps)))


class WarmupCosineSchedule(LambdaLR):
    """ Linear warmup and then cosine decay.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        Decreases learning rate from 1. to 0. over remaining `t_total - warmup_steps` steps following a cosine curve.
        If `cycles` (default=0.5) is different from default, learning rate follows cosine function after warmup.
    """
    def __init__(self, optimizer, warmup_steps, t_total, cycles=.5, last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        self.cycles = cycles
        super(WarmupCosineSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1.0, self.warmup_steps))
        # progress after warmup
        progress = float(step - self.warmup_steps) / float(max(1, self.t_total - self.warmup_steps))
        return max(0.0, 0.5 * (1. + math.cos(math.pi * float(self.cycles) * 2.0 * progress)))


class WarmupCosineWithHardRestartsSchedule(LambdaLR):
    """ Linear warmup and then cosine cycles with hard restarts.
        Linearly increases learning rate from 0 to 1 over `warmup_steps` training steps.
        If `cycles` (default=1.) is different from default, learning rate follows `cycles` times a cosine decaying
        learning rate (with hard restarts).
    """
    def __init__(self, optimizer, warmup_steps, t_total, cycles=1., last_epoch=-1):
        self.warmup_steps = warmup_steps
        self.t_total = t_total
        self.cycles = cycles
        super(WarmupCosineWithHardRestartsSchedule, self).__init__(optimizer, self.lr_lambda, last_epoch=last_epoch)

    def lr_lambda(self, step):
        if step < self.warmup_steps:
            return float(step) / float(max(1, self.warmup_steps))
        # progress after warmup
        progress = float(step - self.warmup_steps) / float(max(1, self.t_total - self.warmup_steps))
        if progress >= 1.0:
            return 0.0
        return max(0.0, 0.5 * (1. + math.cos(math.pi * ((float(self.cycles) * progress) % 1.0))))



class AdamW(Optimizer):
    """ Implements Adam algorithm with weight decay fix.

    Parameters:
        lr (float): learning rate. Default 1e-3.
        betas (tuple of 2 floats): Adams beta parameters (b1, b2). Default: (0.9, 0.999)
        eps (float): Adams epsilon. Default: 1e-6
        weight_decay (float): Weight decay. Default: 0.0
        correct_bias (bool): can be set to False to avoid correcting bias in Adam (e.g. like in Bert TF repository). Default True.
    """
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-6, weight_decay=0.0, correct_bias=True):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {} - should be >= 0.0".format(lr))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[0]))
        if not 0.0 <= betas[1]  < 1.0:
            raise ValueError("Invalid beta parameter: {} - should be in [0.0, 1.0[".format(betas[1]))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {} - should be >= 0.0".format(eps))
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay,
                        correct_bias=correct_bias)
        super(AdamW, self).__init__(params, defaults)

    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')

                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['step'] = 0
                    # Exponential moving average of gradient values
                    state['exp_avg'] = torch.zeros_like(p.data)
                    # Exponential moving average of squared gradient values
                    state['exp_avg_sq'] = torch.zeros_like(p.data)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                # Decay the first and second moment running average coefficient
                # In-place operations to update the averages at the same time
                exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
                exp_avg_sq.mul_(beta2).addcmul_(1.0 - beta2, grad, grad)
                denom = exp_avg_sq.sqrt().add_(group['eps'])

                step_size = group['lr']
                if group['correct_bias']:  # No bias correction for Bert
                    bias_correction1 = 1.0 - beta1 ** state['step']
                    bias_correction2 = 1.0 - beta2 ** state['step']
                    step_size = step_size * math.sqrt(bias_correction2) / bias_correction1

                p.data.addcdiv_(-step_size, exp_avg, denom)

                # Just adding the square of the weights to the loss function is *not*
                # the correct way of using L2 regularization/weight decay with Adam,
                # since that will interact with the m and v parameters in strange ways.
                #
                # Instead we want to decay the weights in a manner that doesn't interact
                # with the m/v parameters. This is equivalent to adding the square
                # of the weights to the loss with plain (non-momentum) SGD.
                # Add weight decay at the end (fixed version)
                if group['weight_decay'] > 0.0:
                    p.data.add_(-group['lr'] * group['weight_decay'], p.data)

        return loss


In [8]:
#@title BERT Model (더블클릭하여 내용 확인 가능합니다.)
logger = logging.getLogger(__name__)

LayerNorm = nn.LayerNorm


def Linear(i_dim, o_dim, bias=True):
    m = nn.Linear(i_dim, o_dim, bias)
    nn.init.normal_(m.weight, std=0.02)
    if bias:
        nn.init.constant_(m.bias, 0.)
    return m


def gelu(x):
    """ Implementation of the gelu activation function currently in Google Bert repo (identical to OpenAI GPT).
        Also see https://arxiv.org/abs/1606.08415
    """
    return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))


def swish(x):
    return x * torch.sigmoid(x)


ACT2FN = {"gelu": gelu, "relu": torch.nn.functional.relu, "swish": swish}


class Attention(nn.Module):
    def __init__(self, config):
        super(Attention, self).__init__()
        self.num_attention_heads = config.num_heads
        self.attention_head_size = int(config.hidden_size / config.num_heads)
        self.all_head_size = self.num_attention_heads * self.attention_head_size

        self.query =Linear(config.hidden_size, self.all_head_size)
        self.key = Linear(config.hidden_size, self.all_head_size)
        self.value = Linear(config.hidden_size, self.all_head_size)

        self.o_proj = Linear(config.hidden_size, config.hidden_size)
        self.dropout = Dropout(config.dropout_prob)

        self.softmax = Softmax(dim=-1)

    def transpose_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        attention_scores = attention_scores + attention_mask
        attention_probs = self.softmax(attention_scores)
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)
        attention_output = self.o_proj(context_layer)

        return attention_output


class PositionWiseFeedForward(nn.Module):
    def __init__(self, config):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = Linear(config.hidden_size, config.ff_dim)
        self.fc2 = Linear(config.ff_dim, config.hidden_size)
        self.act_fn = ACT2FN[config.act_fn]
        self.dropout = Dropout(config.dropout_prob)

    def forward(self, input):
        intermediate = self.fc1(input)
        ff_out = self.dropout(self.fc2(self.act_fn(intermediate)))
        return ff_out


class Block(nn.Module):
    def __init__(self, config):
        super(Block, self).__init__()
        self.attention_norm = LayerNorm(config.hidden_size, eps=1e-12)
        self.ffn_norm = LayerNorm(config.hidden_size, eps=1e-12)
        self.ffn = PositionWiseFeedForward(config)
        self.attn = Attention(config)

    def forward(self, x, attention_mask):
        # Attention
        h = x
        x = self.attn(x, attention_mask)
        x = h + x
        x = self.attention_norm(x)

        # FFN
        h = x
        x = self.ffn(x)
        x = x + h
        x = self.ffn_norm(x)

        return x


class Encoder(nn.Module):
    def __init__(self, config):
        super(Encoder, self).__init__()
        self.layer = nn.ModuleList()
        for l in range(config.num_hidden_layers):
            layer = Block(config)
            self.layer.append(copy.deepcopy(layer))

    def forward(self, hidden_states, attention_mask):
        all_encoder_layers = []
        for layer_block in self.layer:
            hidden_states = layer_block(hidden_states, attention_mask)
            all_encoder_layers.append(hidden_states)

        return all_encoder_layers


class Config(object):
    def __init__(self,
                 vocab_size_or_config_json_file,
                 act_fn="gelu",
                 hidden_size=768,
                 num_hidden_layers=12,
                 ff_dim=3072,
                 num_heads=12,
                 dropout_prob=0.1,
                 max_position_embeddings=512,
                 type_vocab_size=2,
                 initializer_range=0.02,
                 num_classes=2
                 ):
        if isinstance(vocab_size_or_config_json_file, str):
            with open(vocab_size_or_config_json_file, "r", encoding='utf-8') as reader:
                json_config = json.loads(reader.read())
            for key, value in json_config.items():
                self.__dict__[key] = value
        elif isinstance(vocab_size_or_config_json_file, int):
            self.vocab_size = vocab_size_or_config_json_file
            self.act_fn = act_fn
            self.hidden_size = hidden_size
            self.num_hidden_layers = num_hidden_layers
            self.ff_dim = ff_dim
            self.num_heads = num_heads
            self.dropout_prob = dropout_prob
            self.max_position_embeddings = max_position_embeddings
            self.type_vocab_size = type_vocab_size
            self.initializer_range = initializer_range
            self.num_classes = num_classes
        else:
            raise ValueError("First argument must be either a vocabulary size (int)"
                             "or the path to a pretrained model config file (str)")

    @classmethod
    def from_dict(cls, json_object):
        config = Config(vocab_size_or_config_json_file=-1)
        for key, value in json_object.items():
            config.__dict__[key] = value
        return config

    @classmethod
    def from_json_file(cls, json_file):
        with open(json_file, "r", encoding='utf-8') as reader:
            text = reader.read()
        return cls.from_dict(json.loads(text))

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


class Embeddings(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings.
    """
    def __init__(self, config):
        super(Embeddings, self).__init__()
        self.word_embeddings = Embedding(config.vocab_size, config.hidden_size, padding_idx=0)
        self.position_embeddings = Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = Embedding(config.type_vocab_size, config.hidden_size)

        self.LayerNorm = LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = Dropout(config.dropout_prob)

        self.init_weights(config)

    def init_weights(self, config):
        self.word_embeddings.weight.data.normal_(mean=0.0, std=config.initializer_range)
        self.position_embeddings.weight.data.normal_(mean=0.0, std=config.initializer_range)
        self.token_type_embeddings.weight.data.normal_(mean=0.0, std=config.initializer_range)

    def forward(self, input_ids, token_type_ids=None):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)

        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings


class PredictionHeadTransform(nn.Module):  #
    def __init__(self, config):
        super(PredictionHeadTransform, self).__init__()
        self.dense = Linear(config.hidden_size, config.hidden_size)
        self.act_fn = ACT2FN[config.act_fn]
        self.LayerNorm = LayerNorm(config.hidden_size, eps=1e-12)

    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.act_fn(hidden_states)
        hidden_states = self.LayerNorm(hidden_states)
        return hidden_states


class LMPredictionHead(nn.Module):  #
    def __init__(self, config, embedding_weights):
        super(LMPredictionHead, self).__init__()
        self.transform = PredictionHeadTransform(config)
        self.decoder = Linear(embedding_weights.size(1),
                              embedding_weights.size(0),
                              bias=False)
        self.decoder.weight = embedding_weights
        self.bias = nn.Parameter(torch.zeros(embedding_weights.size(0)))

    def forward(self, hidden_states):
        hidden_states = self.transform(hidden_states)
        hidden_states = self.decoder(hidden_states) + self.bias
        return hidden_states


class PreTrainingHeads(nn.Module):  #
    def __init__(self, config, embedding_weights):
        super(PreTrainingHeads, self).__init__()
        self.predictions = LMPredictionHead(config, embedding_weights)

    def forward(self, sequence_output):
        prediction_scores = self.predictions(sequence_output)

        return prediction_scores


class Pooler(nn.Module):
    def __init__(self, config):
        super(Pooler, self).__init__()
        self.dense = Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()

    def forward(self, hidden_states):
        # We "pool" the model by simply taking the hidden state corresponding
        # to the first token.
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.dense(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output


class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        self.embeddings = Embeddings(config)
        self.encoder = Encoder(config)
        self.pooler = Pooler(config)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)

        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype) # fp16 compatibility
        #extended_attention_mask = extended_attention_mask.to(torch.float32)
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, token_type_ids)
        encoded_layers = self.encoder(embedding_output,
                                      extended_attention_mask)

        sequence_output = encoded_layers[-1]
        pooled_output = self.pooler(sequence_output)
        return sequence_output , pooled_output


In [9]:
#@title MRC Model
# encoding: utf-8

import torch
import torch.nn as nn
from torch.nn import functional as F
from loss import *
from torch.nn.modules import CrossEntropyLoss, BCEWithLogitsLoss

class SingleLinearClassifier(nn.Module):
    def __init__(self, hidden_size, num_label):
        super(SingleLinearClassifier, self).__init__()
        self.num_label = num_label
        self.classifier = nn.Linear(hidden_size, num_label)

    def forward(self, input_features):
        features_output = self.classifier(input_features)
        return features_output


class MultiNonLinearClassifier(nn.Module):
    def __init__(self, hidden_size, num_label, dropout_rate):
        super(MultiNonLinearClassifier, self).__init__()
        self.num_label = num_label
        self.classifier1 = nn.Linear(hidden_size, hidden_size)
        self.classifier2 = nn.Linear(hidden_size, num_label)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, input_features):
        features_output1 = self.classifier1(input_features)
        # features_output1 = F.relu(features_output1)
        features_output1 = F.gelu(features_output1)
        features_output1 = self.dropout(features_output1)
        features_output2 = self.classifier2(features_output1)
        return features_output2
        
class BertQueryNER(nn.Module):
    def __init__(self, config):
        super(BertQueryNER, self).__init__()
        self.loss_type = "bce"
        self.bce_loss = BCEWithLogitsLoss(reduction="none")
        dice_smooth = 1e-8
        self.dice_loss = DiceLoss(with_logits=True, smooth=dice_smooth)
        self.bert = Model(config)

        self.start_outputs = nn.Linear(config.hidden_size, 1)
        self.end_outputs = nn.Linear(config.hidden_size, 1)
        self.span_embedding = MultiNonLinearClassifier(config.hidden_size * 2, 1, 0.3)#config.mrc_dropout)

        self.hidden_size = config.hidden_size

    def forward(self, input_ids, token_type_ids=None, attention_mask=None,start_positions=None,end_positions=None, match_labels=None):
        """
        Args:
            input_ids: bert input tokens, tensor of shape [seq_len]
            token_type_ids: 0 for query, 1 for context, tensor of shape [seq_len]
            attention_mask: attention mask, tensor of shape [seq_len]
        Returns:
            start_logits: start/non-start probs of shape [seq_len]
            end_logits: end/non-end probs of shape [seq_len]
            match_logits: start-end-match probs of shape [seq_len, 1]
        """

        bert_outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

        sequence_heatmap = bert_outputs[0]  # [batch, seq_len, hidden]
        batch_size, seq_len, hid_size = sequence_heatmap.size()

        start_logits = self.start_outputs(sequence_heatmap).squeeze(-1)  # [batch, seq_len, 1]
        end_logits = self.end_outputs(sequence_heatmap).squeeze(-1)  # [batch, seq_len, 1]

        start_extend = sequence_heatmap.unsqueeze(2).expand(-1, -1, seq_len, -1)
        end_extend = sequence_heatmap.unsqueeze(1).expand(-1, seq_len, -1, -1)
        span_matrix = torch.cat([start_extend, end_extend], 3)
        span_logits = self.span_embedding(span_matrix).squeeze(-1)

        if start_positions is not None and end_positions is not None:
            batch_size, seq_len = start_logits.size()

            start_float_label_mask = attention_mask.view(-1).float()
            end_float_label_mask = attention_mask.view(-1).float()
            match_label_row_mask = start_label_mask.bool().unsqueeze(-1).expand(-1, -1, seq_len)
            match_label_col_mask = end_label_mask.bool().unsqueeze(-2).expand(-1, seq_len, -1)
            match_label_mask = match_label_row_mask & match_label_col_mask
            match_label_mask = torch.triu(match_label_mask, 0)  # start should be less equal to end

            float_match_label_mask = match_label_mask.view(batch_size, -1).float()

            if self.loss_type == "bce":
                start_loss = self.bce_loss(start_logits.view(-1), start_labels.view(-1).float())
                start_loss = (start_loss * start_float_label_mask).sum() / start_float_label_mask.sum()
                end_loss = self.bce_loss(end_logits.view(-1), end_labels.view(-1).float())
                end_loss = (end_loss * end_float_label_mask).sum() / end_float_label_mask.sum()
                match_loss = self.bce_loss(span_logits.view(batch_size, -1), match_labels.view(batch_size, -1).float())
                match_loss = match_loss * float_match_label_mask
                match_loss = match_loss.sum() / (float_match_label_mask.sum() + 1e-10)

            else:
                start_loss = self.dice_loss(start_logits, start_labels.float(), start_float_label_mask)
                end_loss = self.dice_loss(end_logits, end_labels.float(), end_float_label_mask)
                match_loss = self.dice_loss(span_logits, match_labels.float(), float_match_label_mask)

            return start_loss, end_loss, match_loss, start_logits, end_logits, span_logits
        
        return start_logits, end_logits, span_logits


학습

In [10]:
#import torch_xla.core.xla_model as xm

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
#device = xm.xla_device()
logger.info("device: {} n_gpu: {}".format(device, n_gpu))

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if n_gpu > 0:
    torch.cuda.manual_seed_all(SEED)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)


12/04/2020 11:11:07 - INFO - __main__ -   device: cuda n_gpu: 1


In [11]:
# Prepare Tokenizer
tokenizer = BertTokenizer(vocab_file, max_len=max_seq_length, do_basic_tokenize=True)
config = Config.from_json_file(model_config)

f = open("/drive/My Drive/colab/bert-ner/dataset/EXOBRAIN_NE_CORPUS_10000.txt","r")
all_data = f.readlines()

m = int(len(all_data)*0.8)
train_examples = all_data[:m]
test_examples = all_data[m:]

train_data = MRCNERDataset(tokenizer,train_examples)
train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)
num_train_optimization_steps = int(len(train_data) / train_batch_size) * num_train_epochs

In [12]:
model = BertQueryNER(config)
model.bert.load_state_dict(torch.load(checkpoint,map_location=torch.device("cpu")))
num_params = count_parameters(model)
logger.info("Total Parameter: %d" % num_params)

12/04/2020 11:11:19 - INFO - __main__ -   Total Parameter: 18130691


In [13]:
model.to(device)
model.train()
global_step = 0
epoch = 0

In [14]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm']
optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay': weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters,
                  lr=learning_rate,
                  eps=adam_epsilon)
scheduler = WarmupLinearSchedule(optimizer,
                                 warmup_steps=num_train_optimization_steps*0.1,
                                 t_total=num_train_optimization_steps)

logger.info("***** Running training *****")
logger.info("  Num orig examples = %d", len(train_examples))
logger.info("  Num split examples = %d", len(train_data))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_optimization_steps)
num_train_step = num_train_optimization_steps

12/04/2020 11:11:30 - INFO - __main__ -   ***** Running training *****
12/04/2020 11:11:30 - INFO - __main__ -     Num orig examples = 8000
12/04/2020 11:11:30 - INFO - __main__ -     Num split examples = 14450
12/04/2020 11:11:30 - INFO - __main__ -     Batch size = 2
12/04/2020 11:11:30 - INFO - __main__ -     Num steps = 144500


In [ ]:
for _ in trange(int(num_train_epochs), desc="Epoch"):
    iter_bar = tqdm(train_dataloader, desc="Train Step(XX/XX) (Mean loss=X.X) (loss=X.X)", leave=False, ncols=1000)
    tr_step, total_loss, mean_loss = 0, 0., 0.
    for step, batch in enumerate(iter_bar):
        batch = tuple(t.to(device) for t in batch)  # multi-gpu does scattering it-self
        tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
        attention_mask = (tokens != 0).long()
        start_loss, end_loss, match_loss, start_logits, end_logits, span_logits = model(tokens, token_type_ids=token_type_ids, attention_mask=attention_mask, start_positions = start_labels, end_positions = end_labels, match_labels=match_labels)
        #(self, input_ids, token_type_ids=None, attention_mask=None,tags=None):
        loss = 1.0 * start_loss + 1.0 * end_loss

        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training
        #if config.gradient_accumulation_steps > 1:
        #    loss = loss / config.gradient_accumulation_steps
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1
        tr_step += 1
        total_loss += loss
        mean_loss = total_loss / tr_step
        iter_bar.set_description("Train Step(%d / %d) (Mean loss=%5.5f) (loss=%5.5f)" % (global_step, num_train_step, mean_loss, loss.item()))
            
    logger.info("***** Saving file *****")
    
    model_check_dir = "bertmrc/korquad_%s_%d_%5.5f.bin" % (model.loss_type,epoch,mean_loss)
    logger.info(model_check_dir)
    output_model_file = os.path.join(output_dir,model_check_dir)

    checkpoint = { 
        'epoch': epoch,
        'mean_loss' : mean_loss,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'lr_sched': scheduler.state_dict()}
    
    torch.save(checkpoint, output_model_file)
    epoch += 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:160: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
12/04/2020 09:50:30 - INFO - __main__ -   ***** Saving file *****
12/04/2020 09:50:30 - INFO - __main__ -   bertmrc/korquad_dice_0_0.96015.bin
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


12/04/2020 10:07:39 - INFO - __main__ -   ***** Saving file *****
12/04/2020 10:07:39 - INFO - __main__ -   bertmrc/korquad_dice_1_0.47157.bin


12/04/2020 10:24:56 - INFO - __main__ -   ***** Saving file *****
12/04/2020 10:24:56 - INFO - __main__ -   bertmrc/korquad_dice_2_0.59663.bin


12/04/2020 10:42:13 - INFO - __main__ -   ***** Saving file *****
12/04/2020 10:42:13 - INFO - __main__ -   bertmrc/korquad_dice_3_0.74255.bin


12/04/2020 10:59:22 - INFO - __main__ -   ***** Saving file *****
12/04/2020 10:59:22 - INFO - __main__ -   bertmrc/korquad_dice_4_0.82752.bin


재학습

In [ ]:
model_check_dir = os.path.join(output_dir,"bertmrc/korquad_bce_25_0.00014.bin")
checkpoint = torch.load(model_check_dir)
#model.load_state_dict(checkpoint['model_state_dict'])
epoch_plus = checkpoint['epoch']
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
scheduler.load_state_dict(checkpoint['lr_sched'])

for _ in trange(int(num_train_epochs), desc="Epoch"):
    iter_bar = tqdm(train_dataloader, desc="Train Step(XX/XX) (Mean loss=X.X) (loss=X.X)", leave=False, ncols=1000)
    tr_step, total_loss, mean_loss = 0, 0., 0.
    for step, batch in enumerate(iter_bar):
        batch = tuple(t.to(device) for t in batch)  # multi-gpu does scattering it-self
        tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
        attention_mask = (tokens != 0).long()
        start_loss, end_loss, match_loss, start_logits, end_logits, span_logits = model(tokens, token_type_ids=token_type_ids, attention_mask=attention_mask, start_positions = start_labels, end_positions = end_labels, match_labels=match_labels)
        #(self, input_ids, token_type_ids=None, attention_mask=None,tags=None):
        loss = 1.0 * start_loss + 1.0 * end_loss

        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training
        #if config.gradient_accumulation_steps > 1:
        #    loss = loss / config.gradient_accumulation_steps
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        global_step += 1
        tr_step += 1
        total_loss += loss
        mean_loss = total_loss / tr_step
        iter_bar.set_description("Train Step(%d / %d) (Mean loss=%5.5f) (loss=%5.5f)" % (global_step, num_train_step, mean_loss, loss.item()))
            
    logger.info("***** Saving file *****")
    
    model_check_dir = "bertmrc/korquad_%s_%d_%5.5f.bin" % (model.loss_type,epoch+epoch_plus,mean_loss)
    logger.info(model_check_dir)
    output_model_file = os.path.join(output_dir,model_check_dir)

    checkpoint = { 
        'epoch': epoch+epoch_plus,
        'mean_loss' : mean_loss,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'lr_sched': scheduler.state_dict()}
    torch.save(checkpoint, output_model_file)
    epoch += 1


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:234: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:160: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
12/03/2020 06:47:52 - INFO - __main__ -   ***** Saving file *****
12/03/2020 06:47:52 - INFO - __main__ -   bertmrc/korquad_bce_25_0.00004.bin
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


12/03/2020 07:12:25 - INFO - __main__ -   ***** Saving file *****
12/03/2020 07:12:25 - INFO - __main__ -   bertmrc/korquad_bce_26_0.00007.bin


12/03/2020 07:36:54 - INFO - __main__ -   ***** Saving file *****
12/03/2020 07:36:54 - INFO - __main__ -   bertmrc/korquad_bce_27_0.00004.bin


12/03/2020 08:01:13 - INFO - __main__ -   ***** Saving file *****
12/03/2020 08:01:13 - INFO - __main__ -   bertmrc/korquad_bce_28_0.00001.bin


12/03/2020 08:25:40 - INFO - __main__ -   ***** Saving file *****
12/03/2020 08:25:40 - INFO - __main__ -   bertmrc/korquad_bce_29_0.00002.bin


12/03/2020 08:49:56 - INFO - __main__ -   ***** Saving file *****
12/03/2020 08:49:56 - INFO - __main__ -   bertmrc/korquad_bce_30_0.00008.bin


12/03/2020 08:55:28 - ERROR - root -   Internal Python error in the inspect module.
Below is the traceback from this internal error.

12/03/2020 08:55:28 - INFO - root -   
Unfortunately, your original traceback can not be constructed.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-34-be8321d5c6d1>", line 25, in <module>
    loss.backward()
  File "/usr/local/lib/python3.6/dist-packages/torch/tensor.py", line 221, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/usr/local/lib/python3.6/dist-packages/torch/autograd/__init__.py", line 132, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception oc

KeyboardInterrupt: ignored

테스트

In [15]:
model_check_dir = os.path.join(output_dir, "bertmrc/korquad_dice_1_0.47157.bin")
tokenizer = BertTokenizer(vocab_file, max_len=max_seq_length, do_basic_tokenize=True)
config = Config.from_json_file(model_config)

predict_batch_size = 1

model = BertQueryNER(config)

model.eval()
model.to(device)

checkpoint = torch.load(model_check_dir, map_location=device)
model.load_state_dict(checkpoint['model'])

num_params = count_parameters(model)
logger.info("Total Parameter: %d" % num_params)

12/04/2020 11:11:36 - INFO - __main__ -   Total Parameter: 18130691


In [16]:
model.eval()
test_data = MRCNERDataset(tokenizer,test_examples)
print(len(test_data))
test_sampler = SequentialSampler(test_data)

test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=predict_batch_size)

In [17]:
from metrics.query_span_f1 import QuerySpanF1  
logger.info("Start evaluating!")

true_data = []
predict_data = []
span_f1 = QuerySpanF1(flat=True)
all_start_logits = []
all_end_logits = []
all_span_logits = []

iter_bar = tqdm(test_dataloader, desc="Evaluating", leave=False, ncols=1000)
for batch in iter_bar:
    batch = tuple(t.to(device) for t in batch)
    tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
    #break
    with torch.no_grad():
        attention_mask = (tokens != 0).long()
        batch_start_logits, batch_end_logits, batch_span_logits = model(tokens, token_type_ids=token_type_ids, attention_mask=attention_mask)
        #print(batch_start_logits)
    for i, example_index in enumerate(token_type_ids):
        start_logits, end_logits, span_logits = batch_start_logits[i], batch_end_logits[i], batch_span_logits[i]
    
    all_start_logits += batch_start_logits
    all_end_logits += batch_end_logits
    all_span_logits += batch_span_logits
    
"""
predict_for_f1 = sum(predict_data, [])
true_for_f1 = sum(true_data,[])
"""

12/04/2020 11:11:42 - INFO - __main__ -   Start evaluating!


'\npredict_for_f1 = sum(predict_data, [])\ntrue_for_f1 = sum(true_data,[])\n'

In [18]:
import pickle

wichi = os.path.join(output_dir, "bertmrc/bce1")

with open(wichi+"a", "wb") as fp:   #Pickling
    pickle.dump(all_start_logits, fp)
print(len(all_start_logits))
with open(wichi+"b", "wb") as fp:   #Pickling
    pickle.dump(all_end_logits, fp)
with open(wichi+"c", "wb") as fp:   #Pickling
    pickle.dump(all_span_logits, fp)
"""

with open(wichi+"a", "rb") as fp:   # Unpickling
   all_start_logits = pickle.load(fp)
with open(wichi+"b", "rb") as fp:   # Unpickling
   all_end_logits = pickle.load(fp)
with open(wichi+"c", "rb") as fp:   # Unpickling
   all_span_logits = pickle.load(fp)
"""

3790


'\n\nwith open(wichi+"a", "rb") as fp:   # Unpickling\n   all_start_logits = pickle.load(fp)\nwith open(wichi+"b", "rb") as fp:   # Unpickling\n   all_end_logits = pickle.load(fp)\nwith open(wichi+"c", "rb") as fp:   # Unpickling\n   all_span_logits = pickle.load(fp)\n'

In [19]:
from sklearn.metrics import f1_score
from metrics.query_span_f1 import QuerySpanF1  

span_f1 = QuerySpanF1(flat=True)
#print(f1_score(true_for_f1,predict_for_f1,average = None))
#print(f1_score(true_for_f1,predict_for_f1,average = 'macro'))
#print(f1_score(true_for_f1,predict_for_f1,average = 'weighted'))
result_set = []
print(len(iter_bar))

iter_bar = tqdm(test_dataloader, desc="Evaluating", leave=False, ncols=1000)
for i, batch in enumerate(iter_bar):
    batch = tuple(t.to(device) for t in batch)
    tokens, token_type_ids, start_labels, end_labels, start_label_mask, end_label_mask, match_labels, sample_idx, label_idx = batch
    
    start_preds, end_preds = all_start_logits[i] > 0, all_end_logits[i] > 0
    start_preds = start_preds.unsqueeze(0)
    end_preds = end_preds.unsqueeze(0)
    
    span_f1_stats = span_f1(start_preds=start_preds, end_preds=end_preds, match_logits=all_span_logits[i],
                        start_label_mask=start_label_mask, end_label_mask=end_label_mask,
                        match_labels=match_labels)
    result_set.append(span_f1_stats)
print(span_f1_stats)
with open(wichi+"result_set", "wb") as fp:   #Pickling
    pickle.dump(result_set, fp)

3790


tensor([0., 1., 1.])


In [20]:
all_counts = torch.stack(result_set).sum(0)
span_tp, span_fp, span_fn = all_counts

span_recall = span_tp / (span_tp + span_fn + 1e-10)
span_precision = span_tp / (span_tp + span_fp + 1e-10)
span_f1 = span_precision * span_recall * 2 / (span_recall + span_precision + 1e-10)
print(span_recall,span_precision,span_f1)
print(span_tp)

tensor(0.5524) tensor(0.4304) tensor(0.4838)
tensor(3439.)


In [21]:
one_test = MRCNERDataset.get_test("광주 동구(청장 임택)는 코로나19로 침체된 전통시장 활성화를 위해 국내 최대 쇼핑축제인 ‘2020 코리아세일페스타’와 연계해 남광주시장에서 다양한 이벤트를 진행한다고 밝혔다.",tokenizer=tokenizer,
                                  query='LA')

tokens, token_type_ids, x = one_test
tokens = tokens.to(device).unsqueeze(0)
token_type_ids = token_type_ids.to(device).unsqueeze(0)
attention_mask = (tokens != 0).long()

with torch.no_grad():
    start_logits, end_logits, span_logits = model(tokens, token_type_ids=token_type_ids, attention_mask=attention_mask)

start_pred = start_logits > 0
end_pred = end_logits > 0
span_pred = span_logits > 0

print(span_pred.size())

start_pred = start_pred.tolist()
end_pred = end_pred.tolist()
span_pred = span_pred.tolist()


print([i for i, x in enumerate(start_pred[0]) if x])
print([i for i, x in enumerate(end_pred[0]) if x])
print(x[38:39])
for i in range(len(span_pred)):
    for j in range(len(span_pred)):
        if span_pred[0][i][j]:
            print(i,j)
            print(x[i:j])
#print(start_logits > 0)
#print(end_logits > 0)
#print(span_logits > 0)


['[CLS]', '지역', '/', '장소', '##와', '지형', '/', '지리', '명칭', '등', '##을', '모두', '포함', '##한다', '.', '지역', '/', '장소', '##는', '장소', '계열', '##로', '표시', '##하', '##고', ',', '지형', '/', '지리', '명칭', '##은', '지리', '계열', '##로', '표시', '##한다', '.', '[SEP]', '광주', '동구', '(', '청장', '임', '##택', ')', '는', '코로나', '##1', '##9', '##로', '침체', '##된', '전통', '##시장', '활성', '##화', '##를', '위해', '국내', '최대', '쇼핑', '##축', '##제', '##인', '‘', '2020', '코리아', '##세', '##일', '##페', '##스타', '’', '와', '연계', '##해', '남', '##광', '##주시', '##장', '##에', '##서', '다양', '##한', '이벤트', '##를', '진행', '##한다', '##고', '밝혔', '##다', '.', '[SEP]']
[2, 254, 403, 1821, 28198, 4215, 403, 3977, 1131, 78, 28144, 428, 422, 792, 9, 254, 403, 1821, 28140, 1821, 1667, 28141, 1686, 28142, 28147, 13, 4215, 403, 3977, 1131, 28151, 3977, 1667, 28141, 1686, 792, 9, 3, 2030, 7558, 24, 23585, 236, 28595, 23, 12, 19131, 28145, 28179, 28141, 8939, 28255, 1099, 11753, 3523, 28228, 28154, 240, 1157, 1002, 8419, 28456, 28187, 28164, 370, 10828, 8433, 28222, 28171, 2844

In [ ]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))
    return results


In [ ]:
from torch.utils.data import Dataset
class MRC2NERInput(object):
    def __init__(self, tokens, query, entity_label, impossible=True, end_position=[], span_position=[], start_position=[], qas_id="0.0"):
        self.tokens = tokens
        self.end_position = end_position
        self.entity_label = entity_label
        self.impossible = impossible
        self.query = query
        self.qas_id = qas_id
        self.span_position = span_position
        self.start_position = start_position

labels = {"PS":"사람은 실존인물, 신화속 인물, 게임/소설의 가상 캐릭터 등을 나타낸다.",
          "LA":"지역/장소와 지형/지리 명칭 등을 모두 포함한다. 지역/장소는 장소 계열로 표시하고, 지형/지리 명칭은 지리 계열로 표시한다.",
          "OG":"기관/단체 명칭을 나타내며, 경제기관, 교육기관, 군사기관, 미디어/방송기관, 스포츠기관, 예술기관, 의료기관, 종교기관, 과학기관, 도서관, 법률기관, 정치/정부기관, 음식 관련 기관, 숙박기관 등으로 세분화된다.",
          "AF":"인공물을 나타낸다. 인공물이란 사람에 의해 창조된 대상물을 말하며 문화재, 건물, 악기, 도로, 무기, 운송수단, 작품명 등이 이에 해당한다. 유의할 점은 인공물 내부에 포함된 세부범주에 해당되지 않는 인공물은 태깅 대상에서 배제한다는 점이다.",
          "DT":"날짜 표현을 나타낸다",
          "TI":"시간 표현을 나타낸다",
          "CV":"문명/문화와 관련된 용어들로 구성되어 있다",
          "AM":"생물은 진핵생물 중에서 동물에 해당되는 것으로, 사람을 제외한 짐승을 대상으로 한다.",
          "PT":"식물 명칭을 나타낸다.",
          "QT":"수량 표현을 나타낸다.",
          "FD":"학문 분야를 나타낸다. 세부 범주로는 인문학, 사회과학, 자연과학, 응용과학으로 구성되어 있다. 더불어 학문 분야는 학파를 포함한다. 과학, 사회과학, 의학, 예술, 철학 관련 학파 및 유파들은 학문분야의 세부 범주로 명시된다.",
          "TR":"이론은 특정 이론, 법칙, 원리 등의 명칭을 의미한다. 이론은 명확히 기술될 수 있는 고유의 이론, 법칙, 원리만을 말하며, 일반적인 용어들은 해당하지 않는다.",
          "EV":"특정 사건/사고 명칭을 나타낸다.",
          "MT":"물질에 대한 표현을 나타낸다.",
          "TM":"위에서 정의된 세부 개체명 이외의 개체명 용어를 포함한다. 색, 방향, 기후지역, 모양, 세포 명칭과 생물의 조직 및 기관, 증상/증세/질병, 약, 전자기기 등 "
}
label_to_idx = {f:i+2 for i, f in enumerate(labels)}
class MRCNERDataset_csv(Dataset):
    def __init__(self, tokenizer, read_data, possible_only=True):
        self.read_data = read_data #csv파일이라고 생각

        self.tokenizer = tokenizer
        self.max_length = max_seq_length
        self.possible_only = possible_only

        alldata = []
        for (example_index, data) in enumerate(self.read_data):
            text = data[0]
            x = tokenizer.tokenize(text)

            onedata = []
            for i in labels:
                onedata.append(MRC2NERInput(tokens = text,query=labels[i],entity_label=i,impossible=True,end_position=[],span_position=[],start_position=[],qas_id=str(example_index)+"."+str(label_to_idx[i])))

            for i in range(1,len(data),2):
                keyword = tokenizer.tokenize(data[i])
                label = data[i+1]
                result = find_sub_list(keyword,text)

                for r in result:
                    onedata[label_to_idx[label.upper()]].start_position.append(r[0])
                    onedata[label_to_idx[label.upper()]].impossible = False
                    onedata[label_to_idx[label.upper()]].end_position.append(r[1])
                    onedata[label_to_idx[label.upper()]].span_position.append([r[0],r[1]])                     
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
            for i in range(len(labels)):
                onedata[i].tokens = x

            alldata = alldata + onedata
        
        ## if impossible
        alldata = [
            x for x in alldata if x.start_position
        ]
        self.alldata = alldata
            
    def __len__(self):
        return len(self.alldata)
    
    def __getitem__(self, item):
        """
        Args:
            item: int, idx
        Returns:
            tokens: tokens of query + context, [seq_len]
            token_type_ids: token type ids, 0 for query, 1 for context, [seq_len]
            start_labels: start labels of NER in tokens, [seq_len]
            end_labels: end labels of NER in tokens, [seq_len]
            label_mask: label mask, 1 for counting into loss, 0 for ignoring. [seq_len]
            match_labels: match labels, [seq_len, seq_len]
            sample_idx: sample id
            label_idx: label id
        """
        data = self.alldata[item]
        tokenizer = self.tokenizer

        query = tokenizer.tokenize(data.query)

        x = ["[CLS]"] + query + ["[SEP]"] + data.tokens + ["[SEP]"]
        new_start_positions = [start+2+len(query) for start in data.start_position]
        new_end_positions = [end+2+len(query) for end in data.end_position]
        
        if len(x) > max_seq_length:
            x = query_tokens[0:max_seq_length]
            print("x exceed max_seq_length",len(x))

        id_tokens = tokenizer.convert_tokens_to_ids(x)
        token_type_ids = [[0] * (len(query)+2)] + [[1] * (len(data.tokens)+1)]
        token_type_ids = sum(token_type_ids, [])
        
        match_labels = torch.zeros([max_seq_length, max_seq_length], dtype=torch.long)
        for start, end in zip(new_start_positions, new_end_positions):
            if start >= max_seq_length or end >= max_seq_length:
                continue
            match_labels[start, end] = 1
        
        sample_idx, label_idx = data.qas_id.split(".")
        sample_idx = torch.LongTensor([int(sample_idx)])
        label_idx = torch.LongTensor([int(label_idx)])   

        start_labels = [(1 if idx in new_start_positions else 0)
                        for idx in range(len(id_tokens))]
        end_labels = [(1 if idx in new_end_positions else 0)
                      for idx in range(len(id_tokens))]

        while len(id_tokens) < max_seq_length:
            id_tokens.append(0)
            token_type_ids.append(0)
            start_labels.append(0)
            end_labels.append(0)
        
        return [
            torch.LongTensor(id_tokens),
            torch.LongTensor(token_type_ids),
            torch.LongTensor(start_labels),
            torch.LongTensor(end_labels),
            torch.LongTensor(token_type_ids),#start_label_mask
            torch.LongTensor(token_type_ids),#end _label_mask
            match_labels,
            sample_idx,
            label_idx
        ]

label_to_idx = {"O":1,"I":2,"PS":3,"LA":4,"OG":5,"AF":6,"DT":7,"TI":8,"CV":9,"AM":10,"PT":11,"QT":12,"FD":13,"TR":14,"EV":15,"MT":16,"TM":17}
label_to_idx2 = {"O":1,"B-PS":2,"B-LA":3,"B-OG":4,"B-AF":5,"B-DT":6,"B-TI":7,"B-CV":8,"B-AM":9,"B-PT":10,"B-QT":11,"B-FD":12,"B-TR":13,"B-EV":14,"B-MT":15,"B-TM":16,
                "I-PS":17,"I-LA":18,"I-OG":19,"I-AF":20,"I-DT":21,"I-TI":22,"I-CV":23,"I-AM":24,"I-PT":25,"I-QT":26,"I-FD":27,"I-TR":28,"I-EV":29,"I-MT":30,"I-TM":31}

def text_to_feature(textlist, tokenizer):
    features = []
    unique_id = 1000000000

    for (example_index, data) in enumerate(textlist):
        text = data[0]
        x = tokenizer.tokenize(text)
        x = ["[CLS]"] + x
        x += ["[SEP]"]
        if len(x) > max_seq_length:
            x = query_tokens[0:max_seq_length]
       
        y = [0] * len(x)
        j = 0

        onedata = []
        start_position = []
        end_position = []

        for i in range(1,len(data),2):
            keyword = tokenizer.tokenize(data[i])
            label = data[i+1]
            result = find_sub_list(keyword,text)

            for r in result:
                k = r[0]+1
                y[r[0]] = label_to_idx[label]
                while k < max_seq_length and k <= r[1]
                    y[k] = label_to_idx["I"]
        
        input_ids = tokenizer.convert_tokens_to_ids(x)
        input_mask = ([1] * len(input_ids))
        segment_ids = ([0]* len(input_ids))
        input_length = len(input_ids)
        #위치정보가 있으면 글자별로 tokenize 해서 tok_to_ori길이만큼 이동하여 사용이 맞으나, 위치가 아닌 특수문자 기반으로 진행
        #doc span으로 sliding window 구현이 맞으나, 일단 길이로 잘라서 진행함 (512이하의 데이터임)
        #문장이 1개이므로 segment_id 필요 없음 // 0이라도 채워야 함
        
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)
            y.append(0)

        features.append(
            NERInputFeatures(
                unique_id=unique_id,
                example_index=example_index,
                tokens=x,
                input_ids=input_ids,
                input_mask=input_mask,
                segment_ids = segment_ids,
                tags = y,
                input_length = input_length
            ))
        unique_id += 1

    return features